# chainerRL
## install and environment
 - Ubuntu 16.04 LTS
```
git clone https://github.com/yyuu/pyenv.git ~/.pyenv
echo 'export PYENV_ROOT="$HOME/.pyenv"' >> ~/.bashrc
echo 'export PATH="$PYENV_ROOT/bin:$PATH"' >> ~/.bashrc
echo 'eval "$(pyenv init -)"' >> ~/.bashrc
source ~/.bashrc
pyenv install -l | grep ana #最新のパッケージを検索
pyenv install anaconda3-5.0.1
pyenv rehash
pyenv global anaconda3-5.0.1
echo 'export PATH="$PYENV_ROOT/versions/anaconda3-5.0.1/bin/:$PATH"' >> ~/.bashrc
source ~/.bashrc
conda update conda
pip install tensorflow keras chainerrl gym chainer
```
 - cpu only

## import library

In [1]:
import chainer
import chainer.functions as F
import chainer.links as L
import chainerrl
import numpy as np

## Game Board
 - init ゲームボードの初期化。各エピソードの開始前に実行
   - self
     - board 9マス in float32
     - winner 勝利者
     - missed 
     - done 完了状態
 - reset resetと同様
 - move 手の配置の実行。配置後に勝敗判定やミス（置けないマスへの配置）、ゲーム終了を判定。
 - check_winner 勝利判定。
 - get_empty_pos 配置可能なマスのインデックスのうち一つをランダムで取得。ランダム打ちさせる時に使用。epsilon greedy用。
 - show ボードの状態を表示。人間との対戦用。
 
 ### Game Board
 |0|1|2|  
 |3|4|5|  
 |6|7|8|  
 

In [2]:
class Board():
    def __init__(self):
        self.board = np.array([0] * 9, dtype=np.float32)
        self.winner = None
        self.missed = False
        self.done   = False
        
    def reset(self):
        self.board = np.array([0] * 9, dtype=np.float32)
        self.winner = None
        self.missed = False
        self.done   = False

    def move(self, act, turn):
        if self.board[act] == 0:
            self.board[act] = turn
            self.check_winner()
        else:
            self.winner = turn*-1
            self.missed = True
            self.done = True

    def check_winner(self):
        # Define Win Condition Patern
        win_conditions = ((0,1,2),(3,4,5),(6,7,8),(0,3,6),(1,4,7),(2,5,8),(0,4,8),(2,4,6))
        for cond in win_conditions:
            if self.board[cond[0]] == self.board[cond[1]] == self.board[cond[2]]:
                if self.board[cond[0]]!=0:
                    self.winner=self.board[cond[0]]
                    self.done = True
                    return
        if np.count_nonzero(self.board) == 9:
            self.winner = 0
            self.done = True

    def get_empty_pos(self):
        # Get empties position on Game Board
        empties = np.where(self.board==0)[0]
        # if there are empty position over one, Computer choice random position
        if len(empties) > 0:
            return np.random.choice(empties)
        else:
            return 0

    def show(self):
        row = " {} | {} | {} "
        hr = "\n-----------\n"
        tempboard = []
        for i in self.board:
            if i == 1:
                tempboard.append("○")
            elif i == -1:
                tempboard.append("×")
            else:
                tempboard.append(" ")
        print((row + hr + row + hr + row).format(*tempboard))

## Random Actor
局所解に陥らないよう、random explorerを行う。

あとで統計情報としてどの程度この関数が使用されたか（DQNが考えた手でなくランダムで返したか）を把握できるように、カウンターをインクリメントする。
Boardのget_empty_posを呼び出して、配置可能なマスを取得し、呼び出し元にreturnする。

In [3]:
#explorer用のランダム関数オブジェクト
class RandomActor:
    def __init__(self, board):
        self.board = board
        self.random_count = 0
        
    def random_action_func(self):
        self.random_count += 1
        return self.board.get_empty_pos()

## Q-Function
ChainerRLでNNの実装。
 - obs_size
   - 環境から得られる情報量
 - n_action
   - DQNで得られるaction数
 - n_hidden_channels
   - 隠れ層の数

In [4]:
#Q関数
class QFunction(chainer.Chain):
    def __init__(self, obs_size, n_actions, n_hidden_channels=81):
        super().__init__(
            l0=L.Linear(obs_size, n_hidden_channels),
            l1=L.Linear(n_hidden_channels, n_hidden_channels),
            l2=L.Linear(n_hidden_channels, n_hidden_channels),
            l3=L.Linear(n_hidden_channels, n_actions))
        
    def __call__(self, x, test=False):
        #-1を扱うのでleaky_reluとした
        h = F.leaky_relu(self.l0(x))
        h = F.leaky_relu(self.l1(h))
        h = F.leaky_relu(self.l2(h))
        return chainerrl.action_value.DiscreteActionValue(self.l3(h))

## Prepare Environment and Agent
### 補足
 - LinearDecayEpsilonGreedy
   - epsilon-greedyのやり方を、コンスタントな値でなく、徐々に減らしていく方式
 - LinearDecayEpsilonGreedy関数
   - random_action_func
     - epsilon-greedyする際に必要な関数を渡す必要がある
     - 自作した関数(random_action_func)の引数は同時に渡せない
     - RandomActor Objectのメンバ変数にBoardの参照を渡して、引数なしに自作した関数を呼ばせる
     - epsilon is decrease from start to end epsilon among decay steps 

Agentはoptimizerやreplay_bufferを共有するP1とP2を生成し互いに戦わせる

In [5]:
b  = Board()        # ボードの準備
ra = RandomActor(b) # explorer用のランダム関数オブジェクトの準備
obs_size  = 9       # 環境の次元数（9つのマス）
n_actions = 9       # 選択の数（最大9マスのどれかを選ぶことができる）

# Q-functionとオプティマイザーのセットアップ
q_func    = QFunction(obs_size, n_actions)
optimizer = chainer.optimizers.Adam(eps=1e-2)
optimizer.setup(q_func)

gamma = 0.95 # 報酬の割引率
# Epsilon-greedyを使ってたまに冒険。50000ステップでend_epsilonとなる
explorer = chainerrl.explorers.LinearDecayEpsilonGreedy(
    start_epsilon=1.0, end_epsilon=0.3, decay_steps=50000,
    random_action_func=ra.random_action_func)

# Experience ReplayというDQNで用いる学習手法で使うバッファ(10^6)
replay_buffer = chainerrl.replay_buffer.ReplayBuffer(capacity=10 ** 6)

# Agentの生成（replay_buffer等を共有する2つ）
agent_p1 = chainerrl.agents.DoubleDQN(
    q_func, optimizer, replay_buffer, gamma, explorer,
    replay_start_size=500, update_interval=1,
    target_update_interval=100)

agent_p2 = chainerrl.agents.DoubleDQN(
    q_func, optimizer, replay_buffer, gamma, explorer,
    replay_start_size=500, update_interval=1,
    target_update_interval=100)

## Game Start
先攻も後攻もAgent自身で、ゲームでは内部的に○×の変わりに自身の手を1、相手の手を-1としてボードに配置していく。
先攻後攻どちらの環境・アクションも学習させるため、ゲーム進行内で符号を出し分けるのではなく、常にボードには1を配置する。

In [6]:
n_episodes = 20000 # learning game num
miss = 0           # miss counter
win  = 0           # win counter
draw = 0           # draw counter

#エピソードの繰り返し実行
for i in range(1, n_episodes + 1):
    b.reset()                         # Boardのリセット
    reward = 0                        # 報酬の初期化
    agents = [agent_p1, agent_p2]     # Agentの作成
    turn   = np.random.choice([0, 1]) # 先攻後攻の選択(0:agent_p1,1:agent_p2)
    last_state = None                # 最後の状態

    while not b.done:
        action = agents[turn].act_and_train(b.board.copy(), reward) # 配置マスの取得
        b.move(action, 1)                                           # 配置を実行(1:○の意) moveは勝利・終了判定も行う
        
        # ゲームが終了した場合
        if b.done == True:
            # 最後に選択した手が勝利を決めた場合
            if b.winner == 1:
                reward =  1  # 報酬を設定する
                win    += 1  # 勝利カウンターを増やす
            # 引き分けた場合
            elif b.winner == 0:
                reward =  0
                draw   += 1  # 引き分けカウンターを増やす
            # 負けた場合
            else:
                reward =  -1
            # ルール上、選択してはいけない手を選択した場合
            if b.missed is True:
                miss   += 1
            
            agents[turn].stop_episode_and_train(b.board.copy(), reward, True) # エピソードを終了して学習
            # 相手もエピソードを終了して学習。相手のミスは勝利として学習しないように
            if agents[1 if turn == 0 else 0].last_state is not None and b.missed is False:
                # 前のターンでとっておいたlast_stateをaction実行後の状態として渡す
                agents[1 if turn == 0 else 0].stop_episode_and_train(last_state, reward*-1, True)
                
        # ゲームが終了していない場合
        else:
            last_state = b.board.copy()   # 学習用にターン最後の状態を退避
            b.board    = b.board * -1     # 継続のときは盤面の値を反転
            turn = 1 if turn == 0 else 0 # ターンの切り替え

    #コンソールに進捗表示
    if i % 100 == 0:
        print("episode:", i, " / rnd:", ra.random_count, " / miss:", miss, " / win:", win, " / draw:", draw, " / statistics:", agent_p1.get_statistics(), " / epsilon:", agent_p1.explorer.epsilon)
        #カウンタの初期化
        miss = 0
        win  = 0
        draw = 0
        ra.random_count = 0
    if i % 10000 == 0:
        # 10000エピソードごとにモデルを保存
        agent_p1.save("result_" + str(i))

print("Training finished.")

episode: 100  / rnd: 765  / miss: 2  / win: 88  / draw: 10  / statistics: [('average_q', 0.11796040232930016), ('average_loss', 0.10419970309939416)]  / epsilon: 0.994512
episode: 200  / rnd: 765  / miss: 3  / win: 83  / draw: 14  / statistics: [('average_q', 0.3387417098760484), ('average_loss', 0.1357199689856354)]  / epsilon: 0.989066
episode: 300  / rnd: 743  / miss: 1  / win: 86  / draw: 13  / statistics: [('average_q', 0.5873966454809595), ('average_loss', 0.12998378565300128)]  / epsilon: 0.98383
episode: 400  / rnd: 709  / miss: 10  / win: 82  / draw: 8  / statistics: [('average_q', 0.7833747800566158), ('average_loss', 0.11772076085455689)]  / epsilon: 0.979168
episode: 500  / rnd: 730  / miss: 6  / win: 78  / draw: 16  / statistics: [('average_q', 0.9074705349858682), ('average_loss', 0.10884458429176386)]  / epsilon: 0.973904
episode: 600  / rnd: 718  / miss: 4  / win: 84  / draw: 12  / statistics: [('average_q', 1.0956668627064297), ('average_loss', 0.13227864266463313)]  /

episode: 4900  / rnd: 548  / miss: 5  / win: 89  / draw: 6  / statistics: [('average_q', 0.8447866480266372), ('average_loss', 0.09296305899266825)]  / epsilon: 0.750464
episode: 5000  / rnd: 529  / miss: 4  / win: 90  / draw: 6  / statistics: [('average_q', 0.8336019065844236), ('average_loss', 0.09288730451604757)]  / epsilon: 0.744318
episode: 5100  / rnd: 558  / miss: 5  / win: 86  / draw: 9  / statistics: [('average_q', 0.8397340651257682), ('average_loss', 0.09089364325388923)]  / epsilon: 0.739082
episode: 5200  / rnd: 533  / miss: 3  / win: 87  / draw: 10  / statistics: [('average_q', 0.8299081065872168), ('average_loss', 0.08528103070034489)]  / epsilon: 0.735274
episode: 5300  / rnd: 514  / miss: 3  / win: 85  / draw: 12  / statistics: [('average_q', 0.8171380291325431), ('average_loss', 0.08859431821011361)]  / epsilon: 0.730192
episode: 5400  / rnd: 525  / miss: 8  / win: 83  / draw: 9  / statistics: [('average_q', 0.8421638839498023), ('average_loss', 0.09058070591173309)]

episode: 9700  / rnd: 363  / miss: 3  / win: 86  / draw: 11  / statistics: [('average_q', 0.6741629318778282), ('average_loss', 0.08434566518596688)]  / epsilon: 0.511064
episode: 9800  / rnd: 393  / miss: 2  / win: 76  / draw: 22  / statistics: [('average_q', 0.6731528678006696), ('average_loss', 0.08555124488774021)]  / epsilon: 0.50629
episode: 9900  / rnd: 347  / miss: 0  / win: 86  / draw: 14  / statistics: [('average_q', 0.6892972632738864), ('average_loss', 0.0888750884400752)]  / epsilon: 0.5015020000000001
episode: 10000  / rnd: 345  / miss: 3  / win: 87  / draw: 10  / statistics: [('average_q', 0.6840811369973638), ('average_loss', 0.08595432798814691)]  / epsilon: 0.49663
episode: 10100  / rnd: 322  / miss: 4  / win: 77  / draw: 19  / statistics: [('average_q', 0.6837358987134553), ('average_loss', 0.08923349938145955)]  / epsilon: 0.491688
episode: 10200  / rnd: 347  / miss: 0  / win: 88  / draw: 12  / statistics: [('average_q', 0.7057333035224204), ('average_loss', 0.08446

episode: 14400  / rnd: 231  / miss: 3  / win: 67  / draw: 30  / statistics: [('average_q', 0.5409521661161119), ('average_loss', 0.08298120317651886)]  / epsilon: 0.3
episode: 14500  / rnd: 201  / miss: 2  / win: 75  / draw: 23  / statistics: [('average_q', 0.5481427085711906), ('average_loss', 0.08048439842976354)]  / epsilon: 0.3
episode: 14600  / rnd: 213  / miss: 2  / win: 72  / draw: 26  / statistics: [('average_q', 0.5436728626508801), ('average_loss', 0.08046040480842459)]  / epsilon: 0.3
episode: 14700  / rnd: 192  / miss: 1  / win: 75  / draw: 24  / statistics: [('average_q', 0.551435393846653), ('average_loss', 0.08282731836565647)]  / epsilon: 0.3
episode: 14800  / rnd: 222  / miss: 3  / win: 74  / draw: 23  / statistics: [('average_q', 0.545460425925594), ('average_loss', 0.0810012077912912)]  / epsilon: 0.3
episode: 14900  / rnd: 217  / miss: 3  / win: 65  / draw: 32  / statistics: [('average_q', 0.544170652923286), ('average_loss', 0.08279962009805948)]  / epsilon: 0.3
ep

episode: 19400  / rnd: 220  / miss: 1  / win: 73  / draw: 26  / statistics: [('average_q', 0.49187403476695335), ('average_loss', 0.07546309042224134)]  / epsilon: 0.3
episode: 19500  / rnd: 236  / miss: 0  / win: 76  / draw: 24  / statistics: [('average_q', 0.4952982545575269), ('average_loss', 0.08055740733998867)]  / epsilon: 0.3
episode: 19600  / rnd: 237  / miss: 0  / win: 75  / draw: 25  / statistics: [('average_q', 0.5051632639641752), ('average_loss', 0.07718463908429912)]  / epsilon: 0.3
episode: 19700  / rnd: 209  / miss: 2  / win: 65  / draw: 33  / statistics: [('average_q', 0.4814896300026317), ('average_loss', 0.07818853796458168)]  / epsilon: 0.3
episode: 19800  / rnd: 225  / miss: 2  / win: 60  / draw: 38  / statistics: [('average_q', 0.48356008582480925), ('average_loss', 0.0762779562558232)]  / epsilon: 0.3
episode: 19900  / rnd: 223  / miss: 0  / win: 62  / draw: 38  / statistics: [('average_q', 0.47823412096129275), ('average_loss', 0.07617691497472975)]  / epsilon: 

## Human Player
まずは人間が打てるようにするためのインターフェイスとして、HumanPlayerなるオブジェクトを作成する。

In [9]:
class HumanPlayer:
    def act(self, board):
        valid = False # 無効値
        
        while not valid:
            try:
                act = input("Please enter 1-9: ")
                act = int(act)
                if act >= 1 and act <= 9 and board[act-1] == 0:
                    valid = True #有効値(0-8を選択した場合)
                    return act-1
                else:
                    print ("Invalid move")
            except Exception as e:
                    print (act +  " is invalid")

## Human VS AI
DQNのエージェントは1、人間は-1になるように固定。
先攻・後攻はエピソード開始前に「AIが先攻かどうか」ということを決定して初回をスキップする/しないを制御している。
その関係で、先攻・後攻に関わらずエージェントは常に○で人間は常に×となる。

In [11]:
human_player = HumanPlayer()
agent_p1.load("result_20000")

for i in range(10):
    b.reset()
    dqn_first = np.random.choice([True, False])
    
    while not b.done:
        # AI
        if dqn_first or np.count_nonzero(b.board) > 0:
            b.show()
            action = agent_p1.act(b.board.copy())
            b.move(action, 1)
            
            # ゲームが終了した場合
            if b.done == True:
                if b.winner == 1:
                    print("DQN Win")
                elif b.winner == 0:
                    print("Draw")
                else:
                    print("DQN Missed")
                agent_p1.stop_episode()
                continue
            else:
                print("***************************")
                
        # Human
        b.show()
        action = human_player.act(b.board.copy())
        b.move(action, -1)
        
        # 終了した場合、結果を表示して止める
        if b.done == True:
            if b.winner == -1:
                print("HUMAN Win")
            elif b.winner == 0:
                print("Draw")
            agent_p1.stop_episode()

print("Test finished.")

   |   |   
-----------
   |   |   
-----------
   |   |   
***************************
   | ○ |   
-----------
   |   |   
-----------
   |   |   
Please enter 1-9: 1
 × | ○ |   
-----------
   |   |   
-----------
   |   |   
***************************
 × | ○ |   
-----------
   | ○ |   
-----------
   |   |   
Please enter 1-9: 8
 × | ○ |   
-----------
   | ○ |   
-----------
   | × |   
***************************
 × | ○ |   
-----------
   | ○ |   
-----------
 ○ | × |   
Please enter 1-9: 3
 × | ○ | × 
-----------
   | ○ |   
-----------
 ○ | × |   
***************************
 × | ○ | × 
-----------
   | ○ |   
-----------
 ○ | × | ○ 
Please enter 1-9: 4
 × | ○ | × 
-----------
 × | ○ |   
-----------
 ○ | × | ○ 
DQN Missed
   |   |   
-----------
   |   |   
-----------
   |   |   
Please enter 1-9: 5
   |   |   
-----------
   | × |   
-----------
   |   |   
***************************
   |   | ○ 
-----------
   | × |   
-----------
   |   |   
Please enter 1-9: 1
 × |   | 